### This is a problem posted on Kaggle.com (https://www.kaggle.com/c/facebook-v-predicting-check-ins) in which we have to predict about future check-ins based on train data, the main challenge is data is very large to train by any classifier such as randomforest, boosting or any other ensemble technique got failed due to memory leak issue, so I have devised a simple solution which gives me 0.45 accuracy over this dataset. 

### My solution is divide the dataset into squares of 800 and then find 3 most popular businesses in that square and then predict based on this popularity, one can see from the code that Prep_xy() function prepares the x,y for 800 square grids and grid_sorted will sort the geological x,y according to most popular businessee in that area. Although this solution is very basic but it gives me a head start to work on this problem. The columns like x,y,time and accuracy were very vaguely defined in this problem so I have to try and hit every possible solution I come across.

In [15]:
import datetime
from heapq import nlargest
from operator import itemgetter
import os
import time
import math
from collections import defaultdict
import pandas as pd

In [3]:
train = pd.read_csv('train.csv/train.csv')

In [4]:
train.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [30]:
def prep_xy(x, y):
    range = 800
    ix = math.floor(range*x/10)
    if ix < 0:
        ix = 0
    if ix >= range:
        ix = range-1

    iy = math.floor(range*y/10)
    if iy < 0:
        iy = 0
    if iy >= range:
        iy = range-1

    return ix, iy

In [64]:
def run_solution():
    print('Preparing data...')
    f = open("train.csv/train.csv", "r")
    f.readline()
    total = 0

    grid = defaultdict(lambda: defaultdict(int))
    grid_sorted = dict()

    # Calc counts
    while 1:
        line = f.readline().strip()
        total += 1

        if line == '':
            break

        arr = line.split(",")
        row_id = arr[0]
        x = float(arr[1])
        y = float(arr[2])
        accuracy = arr[3]
        time = arr[4]
        place_id = arr[5]

        ix, iy = prep_xy(x, y)

        grid[(ix, iy)][place_id] += 1

    f.close()

    # Sort array
    for el in grid:
        grid_sorted[el] = nlargest(3, sorted(grid[el].items()), key=itemgetter(1))

    print('Generate submission...')
    sub_file = os.path.join('submission_' + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv')
    out = open(sub_file, "w")
    f = open("test.csv/test.csv", "r")
    f.readline()
    total = 0
    out.write("row_id,place_id\n")

    while 1:
        line = f.readline().strip()
        total += 1

        if line == '':
            break

        arr = line.split(",")
        row_id = arr[0]
        x = float(arr[1])
        y = float(arr[2])
        accuracy = arr[3]
        time = arr[4]

        out.write(str(row_id) + ',')
        filled = []

        ix, iy = prep_xy(x, y)

        s1 = (ix, iy)
        if s1 in grid_sorted:
            topitems = grid_sorted[s1]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                out.write(' ' + topitems[i][0])
                filled.append(topitems[i][0])

        out.write("\n")

    out.close()
    f.close()


start_time = time.time()
run_solution()
print("Elapsed time overall: %s seconds" % (time.time() - start_time))

Preparing data...
Generate submission...
Elapsed time overall: 469.898797750473 seconds


### My solution of adding other time features